In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [0]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import numpy as np

In [0]:
batch_size = 100
lr = 0.0002

transform = transforms.Compose([transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.MNIST(root='./mnist_data/', train=True,
                               transform=transform, download=True)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,
                                          shuffle=True, drop_last=True)
testset = datasets.MNIST(root='./mnist_data/', train=False,
                               transform=transform, download=True)
testloader = torch.utils.data.DataLoader(testset,batch_size=batch_size,
                                          shuffle=False, drop_last=True)

In [0]:
class Generator(nn.Module):
  def __init__(self, input_size, n_class):
    super(Generator, self).__init__()
    self.fc1 = nn.Linear(input_size,256) # 100, 256
    self.fc2 = nn.Linear(256,512)
    self.fc3 = nn.Linear(512, 1024)
    self.fc4 = nn.Linear(1024, n_class) # 1024, 28*28

  def forward(self, x):
    x = F.leaky_relu(self.fc1(x), 0.2)
    x = F.leaky_relu(self.fc2(x), 0.2)
    x = F.leaky_relu(self.fc3(x), 0.2)
    x = F.tanh(self.fc4(x))

    return x

class Discriminator(nn.Module):
  def __init__(self, input_size, n_class):
    super(Discriminator, self).__init__()
    self.fc1 = nn.Linear(input_size, 1024) # 28*28, 1024
    self.fc2 = nn.Linear(1024,512)
    self.fc3 = nn.Linear(512,256)
    self.fc4 = nn.Linear(256, n_class) # 256, 1

  def forward(self, x):
    x = F.leaky_relu(self.fc1(x), 0.2)
    x = F.dropout(x, 0.3)
    x = F.leaky_relu(self.fc2(x), 0.2)
    x = F.dropout(x, 0.3)
    x = F.leaky_relu(self.fc3(x), 0.2)
    x = F.dropout(x, 0.3)
    x = F.sigmoid(self.fc4(x))

    return x 

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G = Generator(input_size = 100, n_class=28*28).to(device)
D = Discriminator(input_size=28*28, n_class=1).to(device)

In [6]:
G

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=784, bias=True)
)

In [7]:
D

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=1, bias=True)
)

In [0]:
BCE_loss = nn.BCELoss()
G_optimizer = optim.Adam(G.parameters(), lr=lr)
D_optimizer = optim.Adam(D.parameters(), lr=lr)

In [0]:
def D_train(x):
  D.zero_grad()

  x_real, y_real = x.view(-1,784), torch.ones(batch_size, 1)
  x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

  D_output = D(x_real)
  D_real_loss = BCE_loss(D_output, y_real)
  D_real_score = D_output

  z = Variable(torch.randn(batch_size, 100).to(device))
  x_fake, y_fake = G(z), Variable(torch.zeros(batch_size, 1).to(device))

  D_output = D(x_fake)
  D_fake_loss = BCE_loss(D_output, y_fake)
  D_fake_score = D_output

  D_loss = D_real_loss + D_fake_loss
  D_loss.backward()
  D_optimizer.step()

  return D_loss.data.item()

In [0]:
def G_train(x):
  G.zero_grad()

  z = Variable(torch.randn(batch_size, 100).to(device))
  y = Variable(torch.ones(batch_size, 1).to(device))

  G_output = G(z)
  D_output = D(G_output)
  G_loss = BCE_loss(D_output, y)

  G_loss.backward()
  G_optimizer.step()

  return G_loss.data.item()

In [14]:
total_epoch = 100
for epoch in range(1, total_epoch+1):
  D_loss_hist, G_loss_hist = [], []
  for batch_idx, (x, _) in enumerate(trainloader):
    D_loss_hist.append(D_train(x))
    G_loss_hist.append(G_train(x))

  print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % ((epoch), total_epoch, torch.mean(torch.FloatTensor(D_loss_hist)), torch.mean(torch.FloatTensor(G_loss_hist))))
  
  if (epoch+1)%5==0:
    sample = G(Variable(torch.randn(batch_size, 100).to(device)))
    save_image(sample.view(sample.size(0),1,28,28), "/gdrive/My Drive/Pytorch/GAN/"+str(epoch)+'.png')
  

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[1/100]: loss_d: 1.200, loss_g: 1.029
[2/100]: loss_d: 1.201, loss_g: 1.030
[3/100]: loss_d: 1.202, loss_g: 1.017
[4/100]: loss_d: 1.193, loss_g: 1.036
[5/100]: loss_d: 1.213, loss_g: 1.009
[6/100]: loss_d: 1.202, loss_g: 1.037
[7/100]: loss_d: 1.207, loss_g: 1.020
[8/100]: loss_d: 1.220, loss_g: 0.986
[9/100]: loss_d: 1.230, loss_g: 0.963
[10/100]: loss_d: 1.225, loss_g: 0.982
[11/100]: loss_d: 1.227, loss_g: 0.990
[12/100]: loss_d: 1.214, loss_g: 1.001
[13/100]: loss_d: 1.211, loss_g: 1.020
[14/100]: loss_d: 1.221, loss_g: 0.987
[15/100]: loss_d: 1.220, loss_g: 0.996
[16/100]: loss_d: 1.229, loss_g: 0.970
[17/100]: loss_d: 1.226, loss_g: 0.993
[18/100]: loss_d: 1.237, loss_g: 0.964
[19/100]: loss_d: 1.232, loss_g: 0.966
[20/100]: loss_d: 1.230, loss_g: 0.973
[21/100]: loss_d: 1.231, loss_g: 0.975
[22/100]: loss_d: 1.235, loss_g: 0.976
[23/100]: loss_d: 1.238, loss_g: 0.956
[24/100]: loss_d: 1.234, loss_g: 0.971
[25/100]: loss_d: 1.236, loss_g: 0.965
[26/100]: loss_d: 1.240, loss_g: 0